In [1]:
import os
import numpy as np
import pandas as pd

import sys
sys.path.append('/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/algorithms')
# from ce_utils.record import printProgress, Sec_to_M_S_ms
# from ce_utils.video import load_frame, find_clip_idxs, merging, captioning
# from ce_utils.data import batch_idxs, Reshape4torch
# from ce_model.CNN_inference import CNN_model
from ce_utils.video import load_frames, model_pred, find_clip_idxs, get_gradcam, process_and_save
from ce_utils.record import total_sum_time
from ce_model.CNN_inference import CNN_model

import argparse

def parse_arguments():
    
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--model_dir', action="store", type=str, 
                        default='/0 data/video/200713 clip for AI re-screening', help='model directory')
    
    parser.add_argument('--model_name', action="store", type=str, 
                        default='np_binary', help='model name')
    
    parser.add_argument('--network', action="store", type=str, 
                        default='CNN_v1', help='network')
    
    parser.add_argument('--gpu_idx', action="store", type=int, 
                        default=3, help='gpu idx used for training')
    
    parser.add_argument('--frame_root', action="store", type=str, 
                        default='/mnt/disk2/data/private_data/SMhospital/capsule/0 data/video/200917 videos for AI feedback', 
                        help='frame directory')
    
    parser.add_argument('--save_root', action="store", type=str, 
                        default='./np_binary', help='save directory')
    
    
    args = parser.parse_args()
    print('args={}'.format(args))
    
    return args


def main(args):

    model = CNN_model(network = args.network, n_ch = 3, n_cls = 2, 
                      model_dir = args.model_dir, model_name = args.model_name, gpu_idx = args.gpu_idx)


    # readme.txt 읽어옴
    video_log = pd.read_csv(args.frame_root + '/readme.txt', header = 0, delimiter = '|', encoding = "utf-8")


    # 최종 처리 이후 csv 파일에 들어갈 column 이름들 (for statistical analysis)
    record = pd.DataFrame(columns = ['index', 'video', '# of frames', 'video length', 'save frame time', 'load frame time', 
                                     'pred. time', '# of positive frames', '# of clip frames', 'clip length',
                                     'Grad-CAM time','process and save time', 'total time'])

    print('model inference, processing, and saving')

    fps = 5

    for i in range(len(video_log)):
        video_info = video_log.iloc[i]

        idx, video_name, frame_save_time, n_frame = (video_info['index'], 
                                                     video_info['video'], 
                                                     video_info['elapsed time'], 
                                                     video_info['number of frames'])

        print('')
        print(idx, ':', video_name, n_frame, '\n')

        frame_dir = os.path.join(args.frame_root, '{:02d}'.format(idx))

        # 프레임을 numpy 형태로 불러옴 & 걸린 시간도 가져옴
        frames_pre, frame_load_time = load_frames(frame_dir)

        # Binary Model과 이미지 파일을 model_pred_and_time 함수에 넣고 리턴으로 binary model의 예측 결과와 걸린 시간을 가져옴
        # 예측결과는 model의 softmax 결과 값이 담긴 list
        preds, pred_time = model_pred(model, frames_pre)

        # 각각의 예측 모델이 예측한 frame은 softmax의 argmax 결과가 1인 index에 해당하는 frame
        # prediction에 해당하는 index만 가져옴 
        pred_idx = np.where(np.argmax(preds, axis = 1) == 1)[0]
        # prediction index 전후 5프레임씩 더하여, 비디오 클립에 필요한 index를 가져옴
        clip_idx = find_clip_idxs(pred_idx, len(frames_pre))

        # 모델과, 프레임 그리고 grad-cam이 필요한 프레임의 index를 통해 grad cam image를 구함
        cams, cam_time = get_gradcam(model, frames_pre, clip_idx, batch_size = 64, target_layers = ['conv7_2'], target_class = None)


        save_dir = os.path.join(args.save_root, '{:02d}'.format(idx))
        # 원본 frame과 grad-cam frame를 붙여서 아래 위치에 저장함
        processing_time = process_and_save(frame_dir, save_dir, frames_pre, cams, preds, clip_idx)

        total_time = total_sum_time([frame_save_time, frame_load_time, pred_time, cam_time, processing_time])

        # 파일 저장 과정에서 단계별로  소요된 시간 계산
        record.loc[i] = [idx, video_name, n_frame, Sec_to_M_S_ms(n_frame/fps), frame_save_time, frame_load_time, 
                         pred_time, len(pred_idx), len(clip_idx), Sec_to_M_S_ms(len(clip_idx)/fps),
                         cam_time, processing_time, total_time]


    record.to_csv(args.save_root + '/record.csv',encoding='utf-8-sig', index = False)
    
    return None

if __name__ == '__main__':
    # invoke the main function of the script
    main(parse_arguments())
    
    
"""
cd /home/yunseob
source Pytorch/bin/activate

cd /mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/algorithms

python3 model_inf_for_SBEC_video.py --frame_root '/mnt/disk2/data/private_data/SMhospital/capsule/0 data/video/200917 videos for AI feedback' --save_dir '/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/video_analysis' --gpu_idx 3
"""

model: np_binary_0.0001_8_2009101037_199_t_accr_0.9819_t_loss_0.331068_v_accr_0.9400_v_loss_0.373449.pt 

model inference, processing, and saving

1 : Gang dae gue (30423812) 28 Oct 16_1.mpg 19390 

19390 |################################################################################ | 100.0% loading frames
19390: 1212(*16) |###############----------------------------------------------------------------- | 19.1% model prediction

KeyboardInterrupt: 